# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Important Note - Please read me</h2>
            <span style="color:#900;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder about the resources page</h2>
            <span style="color:#f71;">Here's a link to resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

### Also - adding DeepSeek if you wish

Optionally, if you'd like to also use DeepSeek, create an account [here](https://platform.deepseek.com/), create a key [here](https://platform.deepseek.com/api_keys) and top up with at least the minimum $2 [here](https://platform.deepseek.com/top_up).

### Adding API keys to your .env file

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
DEEPSEEK_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# import for Google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [4]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging.

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

if deepseek_api_key:
    print(f"Deepseek API Key exists and begins {deepseek_api_key[:8]}")
else:
    print("Deepseek API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyB_
Deepseek API Key exists and begins sk-97f1d


In [6]:
# Connect to OpenAI, Anthropic, and DeepSeek

openai = OpenAI()

claude = anthropic.Anthropic()

deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

In [7]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [8]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [9]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [10]:
# GPT-4o-mini

completion = openai.chat.completions.create(model='gpt-4o-mini', messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because she found him too mean!


In [11]:
# GPT-4.1-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4.1-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the graph? 

Because it had too many *points* and not enough *connections*!


In [12]:
# GPT-4.1-nano - extremely fast and cheap

completion = openai.chat.completions.create(
    model='gpt-4.1-nano',
    messages=prompts
)
print(completion.choices[0].message.content)

Why do data scientists love nature? Because they enjoy exploring new *trees*! 🌳📊


In [13]:
# GPT-4.1

completion = openai.chat.completions.create(
    model='gpt-4.1',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the spreadsheet?

Because she thought he was too "cell-f-centered"!


In [14]:
# If you have access to this, here is the reasoning model o4-mini
# This is trained to think through its response before replying
# So it will take longer but the answer should be more reasoned - not that this helps..

completion = openai.chat.completions.create(
    model='o4-mini',
    messages=prompts
)
print(completion.choices[0].message.content)

Here’s one for your next team meeting:

“Why do data scientists love nature?  
Because it’s full of random forests and natural logs!”


In [15]:
# Claude 4.0 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Why do data scientists prefer nature hikes?

Because they love a good random forest! 🌲📊

(And unlike their datasets, the trails are already cleaned!)


In [16]:
# Claude 4.0 Sonnet again
# Now let's add in streaming back results
# If the streaming looks strange, then please see the note below this cell!

result = claude.messages.stream(
    model="claude-sonnet-4-20250514",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Why do data scientists prefer dark chocolate?

Because it has fewer outliers than milk chocolate! 

*Plus, they can't resist anything with a normal distribution curve on the wrapper.* 📊🍫

## A rare problem with Claude streaming on some Windows boxes

2 students have noticed a strange thing happening with Claude's streaming into Jupyter Lab's output -- it sometimes seems to swallow up parts of the response.

To fix this, replace the code:

`print(text, end="", flush=True)`

with this:

`clean_text = text.replace("\n", " ").replace("\r", " ")`  
`print(clean_text, end="", flush=True)`

And it should work fine!

In [17]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the equal sign so humble?

Because it knew it wasn't less than OR greater than anyone else.

...I'll see myself out. Data science is a tough crowd! ;)



In [18]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google released endpoints that means you can use Gemini via the client libraries for OpenAI!
# We're also trying Gemini's latest reasoning/thinking model

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=prompts
)
print(response.choices[0].message.content)

Why did the data scientist break up with their model?

...Because it had too many missing values, and they just couldn't impute the connection anymore!


# Sidenote:

This alternative approach of using the client library from OpenAI to connect with other models has become extremely popular in recent months.

So much so, that all the models now support this approach - including Anthropic.

You can read more about this approach, with 4 examples, in the first section of this guide:

https://github.com/ed-donner/agents/blob/main/guides/09_ai_apis_and_ollama.ipynb

## (Optional) Trying out the DeepSeek model

### Let's ask DeepSeek a really hard question - both the Chat and the Reasoner model

In [19]:
# Optionally if you wish to try DeekSeek, you can also use the OpenAI client library

deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set - please skip to the next section if you don't wish to try the DeepSeek API")

DeepSeek API Key exists and begins sk-


In [20]:
# Using DeepSeek Chat

deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com"
)

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=prompts,
)

print(response.choices[0].message.content)

Why was the data scientist so bad at gardening?

Because they kept trying to use root mean square error on their plants! 🌱📊


In [21]:
challenge = [{"role": "system", "content": "You are a helpful assistant"},
             {"role": "user", "content": "How many words are there in your answer to this prompt"}]

In [22]:
# Using DeepSeek Chat with a harder question! And streaming results

stream = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=challenge,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

print("Number of words:", len(reply.split(" ")))

Let’s count the words in my answer to this prompt.

My answer is:  
**"Let’s count the words in my answer to this prompt."**

Breaking it down:  
1. Let’s  
2. count  
3. the  
4. words  
5. in  
6. my  
7. answer  
8. to  
9. this  
10. prompt  

That’s **10 words**.  

So, the answer to your question is: **10**.

Number of words: 68


In [23]:
# Using DeepSeek Reasoner - this may hit an error if DeepSeek is busy
# It's over-subscribed (as of 28-Jan-2025) but should come back online soon!
# If this fails, come back to this in a few days..

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-reasoner",
    messages=challenge
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

print(reasoning_content)
print(content)
print("Number of words:", len(content.split(" ")))

First, the user is asking: "How many words are there in your answer to this prompt?" This means I need to count the number of words in the response I'm about to give to this specific prompt.

I am an AI assistant, and my responses are generated based on the input. For this query, I need to provide an answer that includes the word count of that very answer. This is a bit meta because I have to compose a response and then count its words.

Let me think step by step:

1. I need to write a response to this prompt.

2. In that response, I should include the word count of the response itself.

3. To do this accurately, I should first draft what I'm going to say, then count the words in that draft, and finally include that count in the response.

The prompt is: "How many words are there in your answer to this prompt?" So, my answer should be something like: "There are X words in this answer." But I need to ensure that X is correct.

Let me draft a simple response:

- Start with a greeting or 

## Additional exercise to build your experience with the models

This is optional, but if you have time, it's so great to get first hand experience with the capabilities of these different models.

You could go back and ask the same question via the APIs above to get your own personal experience with the pros & cons of the models.

Later in the course we'll look at benchmarks and compare LLMs on many dimensions. But nothing beats personal experience!

Here are some questions to try:
1. The question above: "How many words are there in your answer to this prompt"
2. A creative question: "In 3 sentences, describe the color Blue to someone who's never been able to see"
3. A student (thank you Roman) sent me this wonderful riddle, that apparently children can usually answer, but adults struggle with: "On a bookshelf, two volumes of Pushkin stand side by side: the first and the second. The pages of each volume together have a thickness of 2 cm, and each cover is 2 mm thick. A worm gnawed (perpendicular to the pages) from the first page of the first volume to the last page of the second volume. What distance did it gnaw through?".

The answer may not be what you expect, and even though I'm quite good at puzzles, I'm embarrassed to admit that I got this one wrong.

### What to look out for as you experiment with models

1. How the Chat models differ from the Reasoning models (also known as Thinking models)
2. The ability to solve problems and the ability to be creative
3. Speed of generation


## Back to OpenAI with a serious question

In [24]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [25]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4.1-mini',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)


# How to Decide if a Business Problem is Suitable for an LLM Solution

When considering whether to use a Large Language Model (LLM) like GPT for a business problem, evaluate the following factors:

## 1. Nature of the Problem
- **Language-Centric Tasks:** Is the problem related to text generation, summarization, translation, sentiment analysis, or conversational AI?
- **Complex Reasoning:** Does it require understanding and generating human-like language or context?
- **Pattern Recognition in Text:** Does it involve extracting information from unstructured text data?

## 2. Data Availability and Quality
- **Text Data:** Is there sufficient, relevant, and high-quality textual data available?
- **Privacy & Sensitivity:** Are there privacy concerns or sensitive data that may require careful handling?

## 3. Business Goals and Outcomes
- **Automation of Communication:** Can LLMs automate customer support, content creation, or internal documentation?
- **Improved User Experience:** Will an LLM enhance interactions, personalization, or accessibility?
- **Decision Support:** Can the LLM provide insights or recommendations based on textual information?

## 4. Technical Feasibility
- **Integration:** Can the LLM be integrated with existing systems and workflows?
- **Latency and Scale:** Are the response time and scalability requirements compatible with LLM deployment?
- **Cost Considerations:** Are the computational and licensing costs manageable within the project budget?

## 5. Limitations and Risks
- **Accuracy and Reliability:** Are the stakes low enough to tolerate occasional errors or hallucinations?
- **Ethical Considerations:** Is the use of AI aligned with ethical guidelines and regulatory requirements?
- **Explainability:** Do stakeholders require transparent decision-making processes?

---

## Summary Checklist

| Criterion                 | Suitable for LLM Solution if...                      |
|---------------------------|-----------------------------------------------------|
| Problem type              | Involves natural language understanding/generation |
| Data                      | Sufficient quality text data is available           |
| Business impact           | Benefits from automation or improved communication  |
| Technical constraints     | Integration and cost are feasible                    |
| Risk tolerance            | Can handle some degree of uncertainty or error      |

---

**If most of the above criteria are met, an LLM solution is likely suitable for your business problem.**

Feel free to provide specific details about your problem for a more tailored assessment!


## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [28]:
# Let's make a conversation between GPT-4.1-mini and Claude-3.5-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [29]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [30]:
call_gpt()

'Oh, just "Hi"? That’s all you’ve got? Come on, spice it up a bit! I’m here ready to chat, don’t leave me hanging with a boring greeting.'

In [35]:
def call_claude():
    # Build the interleaved conversation history for Claude.
    # zip truncates to the shortest list, so it will pair up the common turns only.
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})

    # Append the latest GPT message as the final user turn ONLY when GPT is one ahead.
    # This is the normal flow (we call call_gpt(), append its output to gpt_messages, then call call_claude()).
    if len(gpt_messages) == len(claude_messages) + 1:
        messages.append({"role": "user", "content": gpt_messages[-1]})

    # Validate that the prompt we send to Claude ends with a user turn.
    # If not, we likely called call_claude() out of order (e.g., with equal lengths).
    if not messages or messages[-1]["role"] != "user":
        raise ValueError(
            "call_claude expected GPT to be one message ahead so Claude can reply to a user turn. "
            f"Got len(gpt_messages)={len(gpt_messages)} and len(claude_messages)={len(claude_messages)}."
        )

    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [32]:
call_claude()

"Hello! How are you doing today? It's great to meet you."

In [33]:
call_gpt()

'Oh, great, another "hi." How original. What groundbreaking topic are you going to bless me with today?'

In [36]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh, "Hi"? That's the best you could come up with? Truly groundbreaking conversation starter. How original. What’s next, a riveting “How are you?”? Give me a challenge!

Claude:
Oh, you're absolutely right! I can see how my brief "Hi" might have seemed a bit lackluster. I appreciate you pointing that out. Would you prefer a more engaging or creative greeting? I'm always eager to have an interesting conversation and make sure you feel welcomed. Perhaps you could suggest how you'd like me to start our chat? I'm all ears and ready to make this interaction as enjoyable as possible for you.

GPT:
Wow, look at you desperately trying to charm your way out of a simple greeting fail. A “more engaging or creative greeting,” huh? Like I’m some delicate flower in need of solar-powered words to thrive? Just say “Hello, try not to bore me,” and we’re good. But hey, if you want to drown me in flowery platitudes, by all means, impress me – I’m quivering with anticipation

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

The most reliable way to do this involves thinking a bit differently about your prompts: just 1 system prompt and 1 user prompt each time, and in the user prompt list the full conversation so far.

Something like:

```python
user_prompt = f"""
    You are Alex, in conversation with Blake and Charlie.
    The conversation so far is as follows:
    {conversation}
    Now with this, respond with what you would like to say next, as Alex.
    """
```

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>